In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('Tweets.csv')

In [3]:
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [4]:
data = data[['airline_sentiment', 'text']]

In [5]:
data.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [6]:
data.airline_sentiment.unique()

array(['neutral', 'positive', 'negative'], dtype=object)

In [7]:
data.airline_sentiment.value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [8]:
data_positive = data[data.airline_sentiment == 'positive']
data_negative = data[data.airline_sentiment == 'negative']

In [9]:
data_negative = data_negative.iloc[:len(data_positive)]

In [10]:
len(data_negative)

2363

In [11]:
len(data_positive)

2363

In [12]:
data = pd.concat([data_negative, data_positive])

In [13]:
data.sample(len(data))                                   #乱序

,airline_sentiment,text
2251,negative,@united 4595. We are now going back to the gat...
4041,positive,"@United Bringing your ""A"" game with premium ca..."
10428,positive,@USAirways thank you!!!
1172,negative,@united and those three people were awesome wo...
11839,positive,@USAirways will do. Hoping for a voucher for a...
...,...,...
1343,positive,@united I am impressed with your super-fast re...
2018,positive,@united met with agent. All taken care of. Thx...
366,negative,@VirginAmerica I have 2d and 3d embossed badge...
127,positive,@VirginAmerica completely awesome experience l...


数据预处理

In [14]:
data['review'] = (data.airline_sentiment == 'positive').astype('int')

In [15]:
del data['airline_sentiment']

In [16]:
data

,text,review
3,@VirginAmerica it's really aggressive to blast...,0
4,@VirginAmerica and it's a really big bad thing...,0
5,@VirginAmerica seriously would pay $30 a fligh...,0
15,@VirginAmerica SFO-PDX schedule is still MIA.,0
17,@VirginAmerica I flew from NYC to SFO last we...,0
...,...,...
14623,@AmericanAir Love the new planes for the JFK-L...,1
14625,@AmericanAir Flight 236 was great. Fantastic c...,1
14628,Thank you. “@AmericanAir: @jlhalldc Customer R...,1
14630,@AmericanAir Thanks! He is.,1


In [17]:
import re                                                         #使用正则去除特殊字符

In [18]:
token = re.compile('[A-Za-z]+|[!?,.()$]')

In [19]:
def reg_text(text):
    new_text = token.findall(text)
    new_text = [word.lower() for word in new_text]
    return new_text

In [20]:
data['text'] = data.text.apply(reg_text)

In [21]:
data

,text,review
3,"[virginamerica, it, s, really, aggressive, to,...",0
4,"[virginamerica, and, it, s, a, really, big, ba...",0
5,"[virginamerica, seriously, would, pay, $, a, f...",0
15,"[virginamerica, sfo, pdx, schedule, is, still,...",0
17,"[virginamerica, i, flew, from, nyc, to, sfo, l...",0
...,...,...
14623,"[americanair, love, the, new, planes, for, the...",1
14625,"[americanair, flight, was, great, ., fantastic...",1
14628,"[thank, you, ., americanair, jlhalldc, custome...",1
14630,"[americanair, thanks, !, he, is, .]",1


In [22]:
word_set = set()
for text in data.text:
    for word in text:
        word_set.add(word)

In [23]:
max_word = len(word_set)                       #set数据结构会去除所有重复值
max_word

7101

In [24]:
word_list = list(word_set)          #转换为list列表数据结构

In [25]:
word_list.index('you')

5251

In [26]:
word_index = dict((word, word_list.index(word) + 1) for word in word_list)           #将list转化为字典

In [27]:
word_index

{'inpolite': 1,
 'swapping': 2,
 'el': 3,
 'remembering': 4,
 'jackson': 5,
 'hats': 6,
 'oaflfr': 7,
 'malfunction': 8,
 'ceases': 9,
 'hey': 10,
 'east': 11,
 'raleigh': 12,
 'xbo': 13,
 'sons': 14,
 'abassinet': 15,
 'routing': 16,
 'exact': 17,
 'zf': 18,
 'departs': 19,
 'ciscojimfrench': 20,
 'beautiful': 21,
 'monitoring': 22,
 'randomly': 23,
 'mommy': 24,
 'hqoks': 25,
 'yosjhzmc': 26,
 'luxclark': 27,
 'nader': 28,
 'fresh': 29,
 'hrs': 30,
 'xelbon': 31,
 'substandard': 32,
 'sheila': 33,
 'contacting': 34,
 'squished': 35,
 'have': 36,
 'did': 37,
 'need': 38,
 'throw': 39,
 'npxb': 40,
 'sked': 41,
 'tonite': 42,
 'bostonlogan': 43,
 'leopolds': 44,
 'prize': 45,
 'sadie': 46,
 'expense': 47,
 'more': 48,
 'fulfill': 49,
 'rad': 50,
 'celebrating': 51,
 'tho': 52,
 'registration': 53,
 'joanna': 54,
 'dealing': 55,
 'feels': 56,
 'transpacific': 57,
 'inspiring': 58,
 'suck': 59,
 'helpfulness': 60,
 'mammoth': 61,
 'cake': 62,
 'victoria': 63,
 'card': 64,
 'gesture': 65,

In [28]:
data_value = data.text.apply(lambda x: [word_index.get(word, 0) for word in x])    #获取每个word的value值，没有索引则填充为0

In [29]:
len(data_value.iloc[2])

26

In [30]:
max_length = max(len(x) for x in data_value)              
max_length

40

In [31]:
data_value = keras.preprocessing.sequence.pad_sequences(data_value.values, maxlen = max_length)   #填充所有评论至40个单词

In [32]:
data_value.shape

(4726, 40)

In [33]:
data.review.values

array([0, 0, 0, ..., 1, 1, 1])

# 建立模型

In [34]:
model = keras.Sequential()

Embedding 把文本映射为密集向量的层

In [35]:
model.add(layers.Embedding(max_word, 50, input_length = max_length))

In [36]:
model.add(layers.LSTM(64))

In [37]:
model.add(layers.Dense(1, activation = 'sigmoid'))

In [38]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 50)            355050    
_________________________________________________________________
lstm (LSTM)                  (None, 64)                29440     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 384,555
Trainable params: 384,555
Non-trainable params: 0
_________________________________________________________________


In [39]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc']
)

In [41]:
model.fit(data_value, data.review.values, epochs=10, batch_size=64, validation_split=0.2)

Train on 3780 samples, validate on 946 samples
Epoch 1/10
 128/3780 [>.............................] - ETA: 23s - loss: 0.6913 - acc: 0.6719

InvalidArgumentError:  indices[61,35] = 7101 is not in [0, 7101)
	 [[node sequential/embedding/embedding_lookup (defined at D:\Users\Administrator\Anaconda\envs\tensorflow\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]] [Op:__inference_distributed_function_3452]

Function call stack:
distributed_function
